### Python import

In [1]:
import idx2numpy
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tqdm import tqdm

In [2]:
def ACCURACY(true, pred):
    true = true.astype(int)
    pred = pred.astype(int)
    score = np.mean(true == pred)
    return score

### Load Data

In [3]:
train_image = idx2numpy.convert_from_file('data/train-images-idx3-ubyte')
train_label = idx2numpy.convert_from_file('data/train-labels-idx1-ubyte')

test_image = idx2numpy.convert_from_file('data/t10k-images-idx3-ubyte')
test_label = idx2numpy.convert_from_file('data/t10k-labels-idx1-ubyte')

### Pre-processing

In [4]:
def make_label(label):
    output = np.full(
        (len(label), len(np.unique(label))), 0
    )
    for i, digit in enumerate(label):
        output[i, digit] = 1
    return output

In [5]:
x_train = train_image / 255
y_train = make_label(train_label)
    
idx = list(range(0, len(x_train)))
np.random.shuffle(idx)
x_train = x_train[idx]
y_train = y_train[idx]

x_test = test_image / 255
y_test = test_label

### Tensorflow (DNN)

In [6]:
def set_dnn_input(x_input):
    return x_input.reshape(-1, 28*28)

In [7]:
def create_dnn_model():
    inputs = tf.keras.layers.Input(shape=(784))
    x = tf.keras.layers.Dense(10, activation='sigmoid')(inputs)
    model = tf.keras.models.Model(inputs=inputs, outputs=x)    
    model.summary()
    return model

In [8]:
model = create_dnn_model()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 784)]             0         
_________________________________________________________________
dense (Dense)                (None, 10)                7850      
Total params: 7,850
Trainable params: 7,850
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.compile(loss='mse', optimizer='sgd', metrics=['accuracy'])
history = model.fit(set_dnn_input(x_train), y_train, epochs=1, batch_size=1)

Train on 60000 samples
60000/60000 [==============================] - 37s 613us/sample - loss: 0.0419 - accuracy: 0.7991


In [10]:
pred = model.predict(set_dnn_input(x_test))
pred = np.argmax(pred, axis=1)

true = y_test
print('Accuracy: %.2f %%'%(ACCURACY(true, pred)*100))

Accuracy: 87.52 %


### Tensorflow (CNN)

In [11]:
def set_cnn_input(x_input):
    return x_input.reshape(-1, 28, 28, 1)

In [12]:
def create_cnn_model():
    inputs = tf.keras.layers.Input(shape=(28, 28, 1))
    x = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu')(inputs)
    x = tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=None, padding='valid')(x)
    x = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu')(x)
    x = tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=None, padding='valid')(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    x = tf.keras.layers.Dense(10, activation='sigmoid')(x)
    model = tf.keras.models.Model(inputs=inputs, outputs=x)
    model.summary()
    return model

In [13]:
model = create_cnn_model()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 3136)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               4015

In [14]:
model.compile(loss='mse', optimizer='sgd', metrics=['accuracy'])
history = model.fit(set_cnn_input(x_train), y_train, epochs=1, batch_size=1)

Train on 60000 samples
60000/60000 [==============================] - 123s 2ms/sample - loss: 0.0230 - accuracy: 0.8667


In [15]:
pred = model.predict(set_cnn_input(x_test))
pred = np.argmax(pred, axis=1)

true = y_test
print('Accuracy: %.2f %%'%(ACCURACY(true, pred)*100))

Accuracy: 95.08 %
